In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import csv
import os
from tqdm import tqdm  # để hiển thị tiến trình
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
base_url = 'https://mogi.vn/ha-noi/mua-nha'
respone = requests.get(base_url)


In [6]:
Link_list = []
OUTPUT_FILE = "mogi_links.csv"
BATCH_SIZE = 10
MAX_PAGE = 1080

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

def fetch_links(page):
    url = f"{base_url}?&cp={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        return [a.get("href") for a in soup.find_all("a", class_="link-overlay") if a.get("href")]
    except requests.exceptions.RequestException:
        return []

write_header = not os.path.exists(OUTPUT_FILE) or os.path.getsize(OUTPUT_FILE) == 0
with open(OUTPUT_FILE, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    if write_header:
        writer.writerow(["link"])

    for batch_start in tqdm(range(1, MAX_PAGE + 1, BATCH_SIZE), desc="Đang tải trang"):
        batch_end = min(batch_start + BATCH_SIZE - 1, MAX_PAGE)
        with ThreadPoolExecutor(max_workers=BATCH_SIZE) as executor:
            futures = {executor.submit(fetch_links, page): page for page in range(batch_start, batch_end + 1)}
            for future in as_completed(futures):
                links = future.result()
                if links:
                    for href in links:
                        writer.writerow([href])
                        Link_list.append(href)
                    f.flush()
        time.sleep(random.uniform(2, 5))

print(f"Đã tải {len(Link_list)} liên kết từ {base_url} và lưu vào {OUTPUT_FILE}")


Đang tải trang: 100%|██████████| 108/108 [12:32<00:00,  6.97s/it]

Đã tải 16080 liên kết từ https://mogi.vn/ha-noi/mua-nha và lưu vào mogi_links.csv


In [8]:
# -------------------------------
# Cấu hình
# -------------------------------
BATCH_SIZE = 100
INPUT_FILE = 'mogi_links.csv'
OUTPUT_FILE = 'batdongsan_nhadat.csv'
MAX_WORKERS = 10  # số luồng song song (10 là hợp lý)

# -------------------------------
# Đọc danh sách link
# -------------------------------
Link_list = []
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        link = row.get('link')
        if link:
            Link_list.append(link)

print(f"Đã đọc {len(Link_list)} link từ file {INPUT_FILE}.")

# -------------------------------
# Hàm xử lý 1 link (được chạy song song)
# -------------------------------
def crawl_one(link):
    try:
        time.sleep(random.uniform(0.5, 1.5))  # tránh spam server
        res = requests.get(link, timeout=10)
        if res.status_code != 200:
            return None

        soup = BeautifulSoup(res.text, "html.parser")
        data = {}

        price = soup.find('div', class_="price")
        address = soup.find('div', class_="address")
        infos = soup.find_all("div", class_="info-attr clearfix")

        data['price'] = price.get_text(strip=True) if price else None
        data['address'] = address.get_text(strip=True) if address else None

        for info in infos:
            span = info.find_all('span')
            if len(span) >= 2:
                key = span[0].get_text(strip=True)
                value = span[1].get_text(strip=True)
                data[key] = value

        return data
    except Exception as e:
        print(f"Lỗi khi xử lý {link}: {e}")
        return None

# -------------------------------
# Hàm xử lý 1 batch (đa luồng)
# -------------------------------
def process_batch(batch_links):
    batch_data = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(crawl_one, link): link for link in batch_links}
        for future in tqdm(as_completed(futures), total=len(batch_links), desc="  → Crawl song song"):
            result = future.result()
            if result:
                batch_data.append(result)
    return batch_data

# -------------------------------
# Ghi dữ liệu theo batch
# -------------------------------
write_header = not os.path.exists(OUTPUT_FILE) or os.path.getsize(OUTPUT_FILE) == 0
fieldnames = set()

total_batches = (len(Link_list) + BATCH_SIZE - 1) // BATCH_SIZE
for batch_index in range(total_batches):
    start = batch_index * BATCH_SIZE
    end = start + BATCH_SIZE
    batch_links = Link_list[start:end]

    print(f"\nĐang xử lý batch {batch_index + 1}/{total_batches} ({len(batch_links)} link)...")

    batch_data = process_batch(batch_links)

    # Cập nhật fieldnames
    for d in batch_data:
        fieldnames.update(d.keys())

    with open(OUTPUT_FILE, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=list(fieldnames))
        if write_header:
            writer.writeheader()
            write_header = False
        writer.writerows(batch_data)

    print(f" ✅ Đã lưu batch {batch_index + 1}/{total_batches} ({len(batch_data)} bản ghi).")
    time.sleep(random.uniform(3, 5))  # nghỉ giữa các batch

print("\n Hoàn thành! Dữ liệu đã lưu vào", OUTPUT_FILE)

Đã đọc 16080 link từ file mogi_links.csv.

Đang xử lý batch 1/161 (100 link)...


  → Crawl song song: 100%|██████████| 100/100 [00:22<00:00,  4.51it/s]


 ✅ Đã lưu batch 1/161 (100 bản ghi).

Đang xử lý batch 2/161 (100 link)...


  → Crawl song song: 100%|██████████| 100/100 [00:25<00:00,  3.91it/s]


 ✅ Đã lưu batch 2/161 (100 bản ghi).

Đang xử lý batch 3/161 (100 link)...


...

 ✅ Đã lưu batch 159/161 (100 bản ghi).

Đang xử lý batch 160/161 (100 link)...


  → Crawl song song: 100%|██████████| 100/100 [00:19<00:00,  5.22it/s]


 ✅ Đã lưu batch 160/161 (100 bản ghi).

Đang xử lý batch 161/161 (80 link)...


  → Crawl song song: 100%|██████████| 80/80 [00:14<00:00,  5.35it/s]


 ✅ Đã lưu batch 161/161 (80 bản ghi).

 Hoàn thành! Dữ liệu đã lưu vào batdongsan_nhadat.csv
